In [1]:
#conda install -c conda-forge poppler
import pdf2image 

doc_img = pdf2image.convert_from_path("crystal23.pdf", dpi=300)

# print one page as example
doc_img[35]

In [ ]:
import pytesseract 

doc_txt = []
for page in doc_img:
    text = pytesseract.image_to_string(page)
    doc_txt.append(text)

doc_txt[35]